In [52]:
from collections import defaultdict
import math
import pandas as pd
import re
from datetime import datetime 
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
import pylab

print('numpy: ',np.version.version)
print('pandas: ',pd.__version__)
languages = ['en', 'es', 'eu', 'ca', 'pt', 'gl']

numpy:  1.17.2
pandas:  1.0.3


In [53]:
df = pd.read_csv('../OriginalDataSet/training-tweets.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=18000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]

b'Skipping line 17: expected 4 fields, saw 5\nSkipping line 33: expected 4 fields, saw 5\nSkipping line 44: expected 4 fields, saw 5\nSkipping line 58: expected 4 fields, saw 5\nSkipping line 61: expected 4 fields, saw 5\nSkipping line 64: expected 4 fields, saw 5\nSkipping line 74: expected 4 fields, saw 5\nSkipping line 91: expected 4 fields, saw 5\nSkipping line 99: expected 4 fields, saw 5\nSkipping line 140: expected 4 fields, saw 5\nSkipping line 145: expected 4 fields, saw 5\nSkipping line 148: expected 4 fields, saw 5\nSkipping line 152: expected 4 fields, saw 5\nSkipping line 171: expected 4 fields, saw 5\nSkipping line 178: expected 4 fields, saw 5\nSkipping line 197: expected 4 fields, saw 5\nSkipping line 201: expected 4 fields, saw 5\nSkipping line 203: expected 4 fields, saw 5\nSkipping line 213: expected 4 fields, saw 5\nSkipping line 216: expected 4 fields, saw 5\n'


In [54]:
_df0 = df[['Language', 'Tweet']].copy()
pattern = re.compile('[ ]')
# _df0.head()

for language in languages:
    exec("%sAlphabets={}" % (language))
    exec ('{0}Size = 0'.format(language))

In [55]:
start_time = datetime.now() 
trainDict = defaultdict(list)
for index, row in _df0.iterrows():
    sentence = ''
    tweet = row['Tweet']
    language = row['Language']
    for letter in tweet:
        if letter.isalpha(): #or pattern.match(letter):
            exec('if \'{let}\' not in {lang}Alphabets.keys():\n\
                     {lang}Alphabets[letter] = {lang}Size\n\
                     {lang}Size += 1'.format(let=letter, lang=language))#, language, language, language))
            sentence = sentence + letter
        else:
            sentence = sentence + ' '
    print('sentence:',sentence)
    trainDict[row['Language']].append(sentence)
print('Reading time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

sentence: Pedazo de tarta k me e kurrao     Ske k buena soy en todo lo k hago      Jajajajaja http   t co OeSLf D Xl
sentence: No hace falta que te digan que vas a morir para empezar a vivir  
sentence: Empieza mi findeeeee      
sentence: Próxima parada  Carnaval del Toro de Ciudad Rodrigo
sentence: Graniza y tal 
sentence:  Mi idolo  Lo que mas quiero en este mundo  http   t co  xyXrhzAil
sentence: Graniza y luego sale el sol  Muy normal en Irun  si 
sentence: IM CONFUSED  WHAT IS THE REAL JUSTIN   justinbieber  jbbstratfordboy  jdbstratfordboy
sentence:  Nyahika Creo que esta el dependiente majo que tu dijiste 
sentence: Abraham Mateo debe de sentirse querido  http   t co orhSxg y g
sentence:  doolbieber doolly sigueme en insta   que yo te sigo    iker tome  kiss
sentence: Vaya golazo de Gaitan del benfica  Madre mía    
sentence:  GaleHunterJkun Como te salgas ahora te mataré 
sentence: mañana peluquería   esto es vida señores
sentence: Porque estoy raro  Porque no te digo cosas bo

sentence:  carigarcia   zapatillas   pijamas y bragas jajaja
sentence:  eritzurrosolo   garcia maider what were you thinking  http   t co hYFDzMXtvr zezeze  Jajaja
sentence: Qué inutilidad d Ministerio del Interior
sentence: Huelo a el y pf 
sentence: Odio la yuka   
sentence: Haha    de que parte de Rd vienes siendo ventura   RT   Kingventura    iUnaBaarbiiee Ashhh no soy un copion u u 
sentence:  DESPRENDIMIENTOS nivel ROJO en  GI       pk     OÑATI  DGTGipuzkoa  DGT http   t co zoaOyte   
sentence: Si miras atentamente siempre encuentras un camino   
sentence: Y se hizo el fuego      Oiartzun http   t co FriqzmycPu
sentence: Las cosas estan cambiando  ya nada es igual  mierda 
sentence: Luego tengo q ir a buscar a mi hermana
sentence: Para conocerme mejor yo quisiera irme contigo porque mandas en el mundo aunque les joda decirlo  tu das vida y nosotros no 
sentence: Estar viendo La Sexta Noche y recordarme todo lo que  explico Edurne la semana pasada   
sentence:  común es el crimen

sentence:  DeLuque   mis chicas mas bonitas no pueden ser   AMOR  amp  ROMA http   t co aLPZMAmJcX
sentence: Ohh vaya fide de bocattas y tapas el triunfo la torti de patata con champis y mi truqui secret jejeje   
sentence: Lo malo de los carnavales en Irun la muerte del chico de    años por apuñalamiento  ohh pobrecito él y sy family  
sentence: En la calle
sentence: Ahora a leer los tweets del crack de  DavidNieto  K
sentence: Al mal tiempo buena cara   carnaval        Pausok
sentence: Y cuando no me gusta un hombre y me da cotorra  MePongoEnModoCuRa xD http   t co Fe IFx nSX
sentence:  sazkarate ta euskalduna apaizen pare  Zuriz jantzita ta ostiak partitzen  Lotsagarrie
sentence: Igual ayer no se nos fue de las manos    maiwasobi jajajaajajaajajaj
sentence:  AlmendroLopez Vaya  por lo que hablé con Laura creía que tu Quesería ya estaba disponible para el público    jajajaj
sentence: El mono burgos esta locoo
sentence: No sé porque Ancelotti ha salido con el equipo titular y los dos 

sentence: Eusko Ikaskuntza  Sociedad de Estudios Vascos o Nacionalistas  http   t co VyB ADJro 
sentence: Haha tenemo la misma edad RT  Oye Nathy    iUnaBaarbiiee Ven Akaaa Paraate TU U U TU ers La Mayoor Aquii despues me Quemo    
sentence: Se me quitan las ganas de todo  tio 
sentence: Al menos por el wats parezco super happy con mis jajajas y mis xds 
sentence:  OBRAS nivel ROJO en  AP    pk      DGTGipuzkoa  DGT http   t co W  NbF QSg
sentence: Buenos dias reina  noe vega   feliz lunes  muakkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk  unmillondebesos
sentence: Teniamos de todo  el conde dracula  el vampiro y el vampirito  spiderman  blancanieves y la reina de  http   t co NR SPbUJly
sentence:   Que bonita la vida     Palacio Miramar http   t co mM pjxqVnF
sentence: Dejaaa ver xS RT  Oye Nathy    iUnaBaarbiiee Buscalaaa De Jowell Y randy Pah que Veas 
sentence: Cuando mami friege te hago xD RT  Oye Nathy    iUnaBaarbiiee U U Yo Quiero Sanduii Ehh   C

sentence: Vinimos al mundo de la misma forma de la que nos iremos  salimos de un polvo y en polvo nos convertiremos 
sentence:  uxue   animoooo resakossaaa   
sentence: Menos mal que mañana hay clase teórica de gimnasia  que si no   
sentence: One more follower please 
sentence: Frikis todos los que tengais mañana cole 
sentence: Me felicitan más ahora que en todo el día    
sentence:  Br Magic muchaas graciaas tio  
sentence: En fase de PRE ALERTA por  AlarmaRoja  olatu  de guardia esta noche
sentence:  vbhsnjksafjb gracias    por estos MEJORRES CARNAVALES DEL MUNDO JAJAJA tu y yo   contra cualquier fiesta lo sabes  te quiero
sentence: Para conseguir lo que queremos hai que sacrificar algo para poder conseguirlo
sentence: Ahora mismo me hierve la sangre 
sentence: Me gustaria dormir una eternidad 
sentence: Y cada vez que llaman de puta alarma mi cara es un poema  
sentence:  esther      GPCaster lo colgaran como siempre no  O no se ha grabado 
sentence: Eso de la que se avecina a las

...

sentence: No sé qué es peor  si que tu primo elija el número    o que te lo diga que lo hace por S  Ramos     TerreMoto    no tienes ni idea 
sentence: Dejo    que maaal 
sentence: Cuando se quiere  no hay excusas 
sentence:  undivaga pero pero pero    Esto es real   
sentence: Ajajajajajajaja total  nadie lo lee    asique vivan las albondigassss
sentence: Chupipandi   Pepas crew http   t co Jp  INFPGX
sentence: Pues si q son originales en este sitio    Yo os juro q pedí una ensalada  p   Mesón Do Campo http   t co  A MbmF qk
sentence: Alucino con el twitt de Arbeloa Seguir asi seguir que con el mejor portero en el banquillo vais de culo  Apolla a Casillas anda y n al otro 
sentence: No sé que hago con el móvil  así no hago nada  me rayo y yendo como va me pone de los nervios
sentence:   JuanAguilar  Un estudio muy interesante Juan Me recuerda a este q publiqué http   t co dqSXMnw ou Espero q te resulte interesante  Saludos
sentence: Lo único que me jode es que no me hable joder
senten

sentence: La verdad es que Tweet It  Me está gustando mucho más cuanto más lo uso
sentence:  NasuaLocaa feliz cumple  guapa  Disfrútalos mucho  Un besito   
sentence: Rabia  mucha
sentence: merda merda e mas merda  
sentence:  Que tus latidos  que antes eran de cualquiera  ahora solamente suenan por amor  
sentence:  Y que las ropas  estén sucias o estén rotas  casi nunca están reñidas con tener buen corazón  
sentence:  AffbAlejandra  Writer     Tu misma lo has dicho      Andaba        muahahahaha
sentence:  AinhoaNovo  HelenaNoceda O final que poñía a nota 
sentence:  nachibiou jajaja ya tío  lo que me perdí puta bida 
sentence: Iba siendo hora de tener insta  follow me bicht http   t co fWYerhsUT 
sentence: Lo de zedd es tremendo  ULTRALIVE
sentence: Postureo que por certo non me gusta nada  dato
sentence: Mola que te haga daño la cena y te toque darte una ducha a la   de la mañana    
sentence: Como me aburro   voy a escuchar música  cotillear tweets y dormirme 
sentence:  Rubendd 

sentence: Empezar a hablar con  AnitaNts de A y acabar hablando de Z  
sentence: Ahora voy a hablar con mi novio 
sentence:  Meganrogo To be a champion  is not necessary to win championships  you and your team have played like true champions  congrats from Spain
sentence: Primeira caída do meu móvil  va campeón que non será a última  
sentence: Tranquilos  que aunque os falte una hora  solo se cambia un numero  el tiempo sigue siendo el mismo  calma 
sentence: Non hai ansia e se non hai ansia pois non se pode 
sentence: What the hell had just happen in the end of the latest TVD episode  Who is that shadow   TVD
sentence:  alvarolg    por aqui muriendome del asco  y tu   lt  
sentence: Mirad  hay una cosita que se llaman md vale  Y como me aburro pues podíais desaburrirme ahí y esas cositas   
sentence: Fumo mirando como otro día se acaba 
sentence: Voy a perdonar tus pecados de amor bebiendo hasta olvidarlos  BN
sentence: Es tu sonrisa la que me mata 
sentence: Quiero ser el ron que pa

sentence: Quero volver a Italia 
sentence: Lo que ayer dolió  hoy ya no importa 
sentence:  CarrlaGP shhhh     
sentence: Just posted a photo   Sanxenxo http   t co KUn Vmr pC
sentence: Eres demasiado pesado chiico
sentence: Ay como la quiero joder
sentence: Cualquier día me estalla el corazón en el pecho
sentence: Es increíble cómo una persona te puede cambiar el ánimo en un segundo 
sentence: Mi puto gemelo    mañana no voy a entrenar    y encima con dolor de barriga y cabeza
sentence: Ven donde el olvido no tenga nombre  donde felicidad nos corresponde 
sentence: Me cae tan mal Lisa   los episodios dedicados a ella deberían ser censurado 
sentence:  pedropicapalote Felicidades campeón  que ben os mereces   
sentence: Ponerse al día con One Piece y pensar como cojones hacen para que algo que estaba de puta madre ahora es doblemente mejor
sentence: Lo de las torturas a mienbros de Egunkaria ha sido condenado por Estrasburgo  por q Jordi no dice nada 
sentence: Joder  puto Djokovic  le

sentence: Tu guerra todas las noches  tu tregua cada mañana
sentence: Poongamos de moda la FELICIDAD   
sentence: Aquí manda mi polla  punto
sentence: Tengo tanto sueño que no me apetece acabar lo de Filosofía 
sentence: Teño unhas ganas de facer matemáticas    locas
sentence:       Awesome Free Fonts For Designers  http   t co nsOHs lPez  design  free  fonts
sentence:  Los mejores momentos  Todos los que hablo contigo y más 
sentence: Today s outfit tip    outfit  inspiration  outfittip  black  grey  collage  Zara  mango  pullandbear  http   t co OIGKR HxCp
sentence: Ya solo me queda UConn me voy a quedar sin ojos   
sentence: Que asco da la gilipollas 
sentence: La que menos pinta es la que mas sabe
sentence:  alvarogp     y tú siempre con los cascos de los cojones     
sentence:  BraisOliveira a joderse jajaaaja
sentence: Mirando al futuro sin perder de vista el presente porque asumir retos es de valientes  Empezando en Twitter
sentence:  Sari Swift escribese asi     Jajajajajaja va

In [56]:
for language in languages:
    exec('{lang}Model=np.zeros(shape=(({lang}Size+1),({lang}Size+1),({lang}Size+1)))'.format(lang=language))#, language, language, language))

#### For Bigrams

In [21]:
# start_time = datetime.now()
# delta = 0.5
# for language, tweets in trainDict.items():
#     for tweet in tweets:
#         for i in range(len(tweet)-2):
#             first = tweet[i]
#             second = tweet[i+1]
#             exec('firstIndex = %sAlphabets[first]'%(language))
#             exec('secondIndex = %sAlphabets[second]'%(language))
#             exec('%sModel[firstIndex][secondIndex] += 1'%(language))
# for language in languages:
#     exec('%sModel = np.add(%sModel, delta)'%(language, language)) #this is where smoothing happens
#     exec('%sModelFinal = %sModel/%sModel.sum(axis=1)'%(language, language, language)) #divide all the values by the sum of the row
#     exec('%sModelFinal = np.log10(%sModelFinal)'%(language, language))
# print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [46]:
print(enAlphabets)
print('EN',enModel, enModel.shape)

# print(trainDict)

{'I': 0, 'M': 1, 'C': 2, 'O': 3, 'N': 4, 'F': 5, 'U': 6, 'S': 7, 'E': 8, 'D': 9, 'W': 10, 'H': 11, 'A': 12, 'T': 13, 'R': 14, 'L': 15, 'J': 16, 'j': 17, 'u': 18, 's': 19, 't': 20, 'i': 21, 'n': 22, 'b': 23, 'e': 24, 'r': 25, 'a': 26, 'f': 27, 'o': 28, 'd': 29, 'y': 30, 'p': 31, 'h': 32, 'P': 33, 'm': 34, 'l': 35, 'c': 36, 'Q': 37, 'Z': 38, 'B': 39, 'z': 40, 'v': 41, 'g': 42, 'k': 43, 'w': 44, 'Y': 45, 'x': 46, 'K': 47, 'V': 48, 'X': 49, 'q': 50, 'G': 51, 'á': 52, 'ñ': 53, 'Ú': 54, 'í': 55, 'è': 56, 'ó': 57, 'ü': 58, 'é': 59, 'ç': 60, 'ã': 61, 'à': 62, 'ā': 63, 'ò': 64, 'б': 65, 'а': 66, 'р': 67, 'с': 68, 'е': 69, 'л': 70, 'о': 71, 'н': 72, 'É': 73, 'º': 74, 'Ó': 75}
EN [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 

In [ ]:
# trainDict = {'en': ['this is a boy']}
# enModel[24][29][31]

#### For Trigrams

In [57]:
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-2):
            first = tweet[i]
            second = tweet[i+1]
            third = tweet[i+2]
            if (not first.isspace()) and (not second.isspace()) and (not third.isspace()):
                exec('firstIndex = %sAlphabets[first]'%(language))
                exec('secondIndex = %sAlphabets[second]'%(language))
                exec('thirdIndex = %sAlphabets[third]'%(language))
                exec('%sModel[firstIndex][secondIndex][thirdIndex] += 1'%(language))
            
for language in languages:
    exec('for x in range({lang}Model.shape[0]):\n\
    for y in range({lang}Model.shape[0]):\n\
        sumCol = 0\n\
        for z in range({lang}Model.shape[0]):\n\
            {lang}Model[x][y][z] += delta\n\
            sumCol += {lang}Model[x][y][z]\n\
        for z in range({lang}Model.shape[0]):\n\
            {lang}Model[x][y][z] /= sumCol\n\
                \n\
{lang}Model = np.log10({lang}Model)'.format(lang=language))

print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

Training time (hh:mm:ss.ms) 0:00:37.007852


In [35]:
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
#         print(tweet)
        for i in range(len(tweet) - 2):
            first = tweet[i]
            second = tweet[i + 1]
            third = tweet[i + 2]
    #         print(first,second,third)
            if (not first.isspace()) and (not second.isspace()) and (not third.isspace()):
    #             print(first,second,third)
                exec('firstIndex = %sAlphabets[first]' % language)
                exec('secondIndex = %sAlphabets[second]' % language)
                exec('thirdIndex = %sAlphabets[third]' % language)
                exec('%sModel[firstIndex][secondIndex][thirdIndex] += 1' % language)
for language in languages:
    exec('for x in range({L}Model.shape[0]):\n\
    for y in range({L}Model.shape[1]):\n\
        {L}ModelTemp = {L}Model[x,y,:]\n\
        {L}ModelTemp = np.add({L}ModelTemp, delta)\n\
        {L}ModelTemp = np.divide({L}ModelTemp, {L}ModelTemp.sum(axis=0))\n\
        {L}ModelTemp = np.log10({L}ModelTemp)\n\
        {L}Model[x,y,:] = {L}ModelTemp\n'.format(L=language))
print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

Training time (hh:mm:ss.ms) 0:00:28.587039


In [ ]:
print(enModel)#, file=open("enModel.txt", "a"))

## Testing the models

In [58]:
file = open('filename1.txt', 'w')
print('TWEETID', '  ', 'PREDICTEDVALUE', '  ', 'PROBABILITY', '  ', 'ACTUALVALUE', 'RESULT', file=file, end='\n')
df = pd.read_csv('../OriginalDataSet/test-tweets-given.txt', encoding='utf-8', error_bad_lines=False, sep='\t')#, nrows=1000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]
_df0 = df[['TweetID', 'Language', 'Tweet']].copy()
# pattern = re.compile('[a-z ]')
# _df0.head()

In [59]:
probability = {}
for index, row in _df0.iterrows():
    for language in languages:
        exec("%sProb=math.log10(1/6)" % (language))
    tweetID = row['TweetID']
    langTweet = row['Language']
    tweet = row['Tweet']
    
    for i in range(len(tweet)-2):
        first = tweet[i]
        second = tweet[i+1]
        third = tweet[i+2]
        for language in languages:
            exec('if ( (not first.isalpha()) and (not second.isalpha()) and (not third.isalpha()) ):\n\
    prob = 0 \n\
elif ((first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys())):\n\
    prob = {lang}Model[-1][-1][-1]\n\
elif ( (first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][-1][index]\n\
elif ( (first not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[second]\n\
    prob = {lang}Model[-1][index][-1]\n\
elif ( (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[first]\n\
    prob = {lang}Model[index][-1][-1]\n\
elif first not in {lang}Alphabets.keys():\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][secondIndex][thirdIndex]\n\
elif second not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][-1][thirdIndex]\n\
elif third not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    prob = {lang}Model[firstIndex][secondIndex][-1]\n\
else:\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][secondIndex][thirdIndex]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang=language))
    for langu in languages:
        exec("probability['%s'] = %sProb"%(langu, langu))
    result = max(probability, key=probability.get)
    print(tweetID, '  ', result, '  ', '%.2E' % Decimal(probability[result]), '  ', langTweet, 'correct' if (langTweet==result) else 'wrong', file=file, end='\n')

# Important Stuff

In [ ]:
delta=1
B = np.arange(27.0).reshape((3,3,3))
print(B)
B[0][0][0] = 0
B[0][0][1] = 0
B[0][0][2] = 0
for x in range(B.shape[0]):
    for y in range(B.shape[0]):
        colSlice = B[x,y,:]
        colSlice = np.add(colSlice, delta)
        colSlice = colSlice/colSlice.sum()
        sumCol = 0
        for z in range(B.shape[0]):
            sumCol = sumCol + B[x][y][z]
#         print('sum for [%d][%d]'%(x,y), sumCol)
        if sumCol != 0:
            for z in range(B.shape[0]):
                B[x][y][z] /= sumCol
print(B)

In [ ]:
language = 'en'
str = 'for x in range({lang}Model.shape[0]):\n\
    for y in range({lang}Model.shape[0]):\n\
        {lang}ModelTemp = {lang}Model[x,y,:]\n\
        {lang}ModelTemp = np.add({lang}ModelTemp, delta)\n\
        {lang}ModelTemp = np.divide({lang}ModelTemp, {lang}Model.sum())\n\
        {lang}ModelTemp = np.log10({lang}ModelTemp)\n\
        {lang}Model[x,y,:] = {lang}ModelTemp\n\
{lang}Model = np.log10({lang}Model)'.format(lang=language)
print(str)

In [ ]:
lang = 'en'
str = 'if ( (not first.isalpha()) and (not second.isalpha()) and (not third.isalpha()) ):\n\
    prob = 0 \n\
elif ((first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys())):\n\
    prob = {lang}Model[-1][-1][-1]\n\
elif ( (first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][-1][index]\n\
elif ( (first not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[second]\n\
    prob = {lang}Model[-1][index][-1]\n\
elif ( (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[first]\n\
    prob = {lang}Model[index][-1][-1]\n\
elif first not in {lang}Alphabets.keys():\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][secondIndex][thirdIndex]\n\
elif second not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][-1][thirdIndex]\n\
elif third not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    prob = {lang}Model[firstIndex][secondIndex][-1]\n\
else:\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][secondIndex][thirdIndex]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang='en')
    
print(str)

In [51]:
Model = np.array([])
Model = np.resize(Model, (enSize+1))
Model.shape

(77,)